In [1]:
import tkinter as tk
from tkinter import filedialog
import glob
from pathlib import Path
import h5py
import numpy as np
from PIL import Image, ImageOps
import cv2

In [2]:
FOLDER_PATH = "/mnt/d/Datasets/PVDN/images/"
IMAGES_LIST = glob.glob(f"{FOLDER_PATH}/*.png")
print(len(IMAGES_LIST))

59746


In [ ]:
def isgray(imgpath):
    img = cv2.imread(imgpath)
    if len(img.shape) < 3: return True
    if img.shape[2]  == 1: return True
    b,g,r = img[:,:,0], img[:,:,1], img[:,:,2]
    if (b==g).all() and (b==r).all(): return True
    return False

print(isgray("/mnt/d/Datasets/PVDN/images/S00000_1.png"))

In [5]:
# Image loaded using cv2, compressed with cv2.imencode, using np resize
import h5py
import numpy as np
from PIL import Image
import cv2
import os

PVDN_DATASET_SIZE = 59746
# PVDN_DATASET_SIZE = 250
PVDN_IMAGE_BYTES_IN_CHUNK = 600000

PVDN_IMAGES_PER_CHUNK = 1

hdf5_path = "/mnt/d/Datasets/PVDN/images_hdf5/images_labels_final.hdf5"

print("Generate HDF5 to %s" % hdf5_path)

# Define the dataset dimensions and chunk sizes
dims = (PVDN_DATASET_SIZE, PVDN_IMAGE_BYTES_IN_CHUNK)
chunk_sizes = (PVDN_IMAGES_PER_CHUNK, PVDN_IMAGE_BYTES_IN_CHUNK)
image_files = IMAGES_LIST

with h5py.File(hdf5_path, "w") as hf:
    pvdn_group = hf.create_group("PVDN")
    pvdn_images_ds = pvdn_group.create_dataset("images", shape=dims, dtype=np.uint8, chunks=chunk_sizes, compression='lzf')
    pvdn_labels_ds = pvdn_group.create_dataset("labels", shape=(PVDN_DATASET_SIZE,), dtype=h5py.special_dtype(vlen=bytes))


    for i, image_file_path in enumerate(image_files):
        if i >= PVDN_DATASET_SIZE:
            break

        # Processing images
        image = cv2.imread(str(image_file_path), cv2.IMREAD_GRAYSCALE)
        _, image_bytes = cv2.imencode('.jpg', image)
        image_bytes = np.resize(image_bytes, (PVDN_IMAGE_BYTES_IN_CHUNK,))
        pvdn_images_ds[i] = image_bytes

        # Read the label text file
        parent_dir = os.path.dirname(os.path.dirname(image_file_path))  # Go up two levels to reach the desired parent directory
        label_file_path = os.path.join(parent_dir, "labels", os.path.splitext(os.path.basename(image_file_path))[0] + ".txt")
       

        with open(label_file_path, "rb") as fileobj:
            bytes_list = fileobj.read().splitlines()
            filtered_bytes_no_other = [byte for byte in bytes_list if not byte.startswith(b'2')]
            # print(label_file_path, filtered_bytes_no_other)
            result = b'\n'.join(filtered_bytes_no_other)
            # print(result)
            pvdn_labels_ds[i] = result
        
        
        if (i + 1) % 1000 == 0:
            print(f"Processed image {i+1} to {min(i + 1000, PVDN_DATASET_SIZE)}")


print("Done processing.")

Generate HDF5 to /mnt/d/Datasets/PVDN/images_hdf5/images_labels_final.hdf5
Processed image 1000 to 1999
Processed image 2000 to 2999
Processed image 3000 to 3999
Processed image 4000 to 4999
Processed image 5000 to 5999
Processed image 6000 to 6999
Processed image 7000 to 7999
Processed image 8000 to 8999
Processed image 9000 to 9999
Processed image 10000 to 10999
Processed image 11000 to 11999
Processed image 12000 to 12999
Processed image 13000 to 13999
Processed image 14000 to 14999
Processed image 15000 to 15999
Processed image 16000 to 16999
Processed image 17000 to 17999
Processed image 18000 to 18999
Processed image 19000 to 19999
Processed image 20000 to 20999
Processed image 21000 to 21999
Processed image 22000 to 22999
Processed image 23000 to 23999
Processed image 24000 to 24999
Processed image 25000 to 25999
Processed image 26000 to 26999
Processed image 27000 to 27999
Processed image 28000 to 28999
Processed image 29000 to 29999
Processed image 30000 to 30999
Processed ima